In [46]:
!pip install memory_profiler
%load_ext memory_profiler

time: 3.43 s (started: 2025-03-23 15:06:51 +00:00)


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install ipython-autotime
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00
time: 273 µs (started: 2025-03-23 14:47:47 +00:00)


In [ ]:
!python --version

Python 3.11.11


In [ ]:
!wget https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz
!rm spark-3.5.3-bin-hadoop3.tgz   # Tidying up

--2025-03-23 13:19:47--  https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400864419 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.3-bin-hadoop3.tgz’

spark-3.5.3-bin-had 100%[===================>] 382.29M  18.8MB/s    in 22s     

2025-03-23 13:20:09 (17.6 MB/s) - ‘spark-3.5.3-bin-hadoop3.tgz’ saved [400864419/400864419]



In [ ]:
# Setting up our environmental variables:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) #  This will format our output tables a bit nicer when not using the show() method
spark

time: 16.2 s (started: 2025-03-23 14:48:01 +00:00)


In [ ]:
import multiprocessing
print(multiprocessing.cpu_count())

2
time: 8.44 ms (started: 2025-03-23 14:48:27 +00:00)


In [ ]:
# Creating our SparkContext:
sc = spark.sparkContext

time: 363 µs (started: 2025-03-23 14:48:28 +00:00)


In [ ]:
%cd /content/sample_data/assignment_data/

/content/sample_data/assignment_data
time: 1.96 ms (started: 2025-03-23 14:50:08 +00:00)


In [ ]:
fin = open('NEISS.TXT', encoding='latin1')
data = fin.read()
fin.close()

time: 440 ms (started: 2025-03-23 14:50:10 +00:00)


In [ ]:
df = spark.read.options(delimiter="\t", header=True, inferSchema=True).csv("/content/sample_data/assignment_data/NEISS.TXT")

time: 17.6 s (started: 2025-03-23 14:50:29 +00:00)


In [ ]:
df.printSchema()

root
 |-- CPSC_Case_Number: string (nullable = true)
 |-- Treatment_Date: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Sex: double (nullable = true)
 |-- Race: integer (nullable = true)
 |-- Other_Race: string (nullable = true)
 |-- Hispanic: string (nullable = true)
 |-- Body_Part: integer (nullable = true)
 |-- Diagnosis: integer (nullable = true)
 |-- Other_Diagnosis: string (nullable = true)
 |-- Body_Part_2: string (nullable = true)
 |-- Diagnosis_2: string (nullable = true)
 |-- Other_Diagnosis_2: string (nullable = true)
 |-- Disposition: integer (nullable = true)
 |-- Location: integer (nullable = true)
 |-- Fire_Involvement: integer (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- Drug: string (nullable = true)
 |-- Product_1: integer (nullable = true)
 |-- Product_2: integer (nullable = true)
 |-- Product_3: integer (nullable = true)
 |-- Narrative: string (nullable = true)
 |-- Stratum: string (nullable = true)
 |-- PSU: integer (nullable = 

In [48]:
df.select("*").show(5)

+----------------+--------------+---+---+----+----------+--------+---------+---------+---------------+-----------+-----------+-----------------+-----------+--------+----------------+-------+----+---------+---------+---------+--------------------+-------+---+--------+
|CPSC_Case_Number|Treatment_Date|Age|Sex|Race|Other_Race|Hispanic|Body_Part|Diagnosis|Other_Diagnosis|Body_Part_2|Diagnosis_2|Other_Diagnosis_2|Disposition|Location|Fire_Involvement|Alcohol|Drug|Product_1|Product_2|Product_3|           Narrative|Stratum|PSU|  Weight|
+----------------+--------------+---+---+----+----------+--------+---------+---------+---------------+-----------+-----------+-----------------+-----------+--------+----------------+-------+----+---------+---------+---------+--------------------+-------+---+--------+
|       160101845|    01/01/2016| 92|1.0|   0|      NULL|    NULL|       79|       57|           NULL|       NULL|       NULL|             NULL|          4|       1|               0|   NULL|NULL| 

In [ ]:
from pyspark.ml.feature import Bucketizer
bins = [0,2,11,21,51,81,121,224]
df2 = Bucketizer(splits=bins,inputCol='Age',outputCol='Age_bins').transform(df)

time: 946 ms (started: 2025-03-23 14:50:55 +00:00)


In [ ]:
df2.select("*").show(5)

+----------------+--------------+---+---+----+----------+--------+---------+---------+---------------+-----------+-----------+-----------------+-----------+--------+----------------+-------+----+---------+---------+---------+--------------------+-------+---+--------+--------+
|CPSC_Case_Number|Treatment_Date|Age|Sex|Race|Other_Race|Hispanic|Body_Part|Diagnosis|Other_Diagnosis|Body_Part_2|Diagnosis_2|Other_Diagnosis_2|Disposition|Location|Fire_Involvement|Alcohol|Drug|Product_1|Product_2|Product_3|           Narrative|Stratum|PSU|  Weight|Age_bins|
+----------------+--------------+---+---+----+----------+--------+---------+---------+---------------+-----------+-----------+-----------------+-----------+--------+----------------+-------+----+---------+---------+---------+--------------------+-------+---+--------+--------+
|       160101845|    01/01/2016| 92|1.0|   0|      NULL|    NULL|       79|       57|           NULL|       NULL|       NULL|             NULL|          4|       1|    

In [ ]:
from pyspark.sql.functions import lit, map_from_arrays, array, col
age_dict = {0:'Not Recorded', 1:'Child', 2:'Adolescent',3:'Adult',4:'Elderly',5:'Very old',6:'Infant'}
keys = array(list(map(lit, age_dict.keys())))
values = array(list(map(lit, age_dict.values())))
age_map = map_from_arrays(keys, values)

time: 50.8 ms (started: 2025-03-23 14:51:00 +00:00)


In [ ]:
df2 = df2.withColumn("Age_Categories", age_map.getItem(col("Age_bins")))

time: 78 ms (started: 2025-03-23 14:51:02 +00:00)


/usr/local/lib/python3.11/dist-packages/pyspark/sql/column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [ ]:
df2.select("*").show(5)

+----------------+--------------+---+---+----+----------+--------+---------+---------+---------------+-----------+-----------+-----------------+-----------+--------+----------------+-------+----+---------+---------+---------+--------------------+-------+---+--------+--------+--------------+
|CPSC_Case_Number|Treatment_Date|Age|Sex|Race|Other_Race|Hispanic|Body_Part|Diagnosis|Other_Diagnosis|Body_Part_2|Diagnosis_2|Other_Diagnosis_2|Disposition|Location|Fire_Involvement|Alcohol|Drug|Product_1|Product_2|Product_3|           Narrative|Stratum|PSU|  Weight|Age_bins|Age_Categories|
+----------------+--------------+---+---+----+----------+--------+---------+---------+---------------+-----------+-----------+-----------------+-----------+--------+----------------+-------+----+---------+---------+---------+--------------------+-------+---+--------+--------+--------------+
|       160101845|    01/01/2016| 92|1.0|   0|      NULL|    NULL|       79|       57|           NULL|       NULL|       NUL

In [ ]:
col_list = ['Body_Part_2', 'Diagnosis_2', 'Other_Diagnosis_2', 'Location', 'Fire_Involvement', 'Alcohol', 'Drug', 'Product_1', 'Product_2', 'Product_3', 'Narrative', 'Stratum', 'PSU', 'Weight', 'Age_bins']
df2 = df2.drop(*col_list)

time: 19.3 ms (started: 2025-03-23 14:51:07 +00:00)


In [ ]:
df2.select("*").show(5)

+----------------+--------------+---+---+----+----------+--------+---------+---------+---------------+-----------+--------------+
|CPSC_Case_Number|Treatment_Date|Age|Sex|Race|Other_Race|Hispanic|Body_Part|Diagnosis|Other_Diagnosis|Disposition|Age_Categories|
+----------------+--------------+---+---+----+----------+--------+---------+---------+---------------+-----------+--------------+
|       160101845|    01/01/2016| 92|1.0|   0|      NULL|    NULL|       79|       57|           NULL|          4|      Very old|
|       160101847|    01/01/2016| 90|1.0|   0|      NULL|    NULL|       79|       57|           NULL|          4|      Very old|
|       160101848|    01/01/2016| 71|2.0|   0|      NULL|    NULL|       79|       57|           NULL|          4|       Elderly|
|       160101852|    01/01/2016| 71|2.0|   0|      NULL|    NULL|       31|       64|           NULL|          1|       Elderly|
|       160101857|    01/01/2016| 57|1.0|   0|      NULL|    NULL|       83|       59|    

In [ ]:
df2 = df2.where(df2['Disposition'] == 8)

time: 21.1 ms (started: 2025-03-23 14:51:10 +00:00)


In [ ]:
df2.select("*").show(5)

+----------------+--------------+---+---+----+----------+--------+---------+---------+--------------------+-----------+--------------+
|CPSC_Case_Number|Treatment_Date|Age|Sex|Race|Other_Race|Hispanic|Body_Part|Diagnosis|     Other_Diagnosis|Disposition|Age_Categories|
+----------------+--------------+---+---+----+----------+--------+---------+---------+--------------------+-----------+--------------+
|       160122220|    01/08/2016| 84|1.0|   0|      NULL|    NULL|       85|       71|      CARDIAC ARREST|          8|      Very old|
|       160127991|    01/09/2016| 41|2.0|   1|      NULL|    NULL|       85|       71|CARDIOPULMONARY ARRE|          8|         Adult|
|       160135106|    01/17/2016| 23|2.0|   3|  HISPANIC|    NULL|       75|       52|                NULL|          8|         Adult|
|       160135933|    01/10/2016| 58|2.0|   1|      NULL|    NULL|       85|       71|        SUDDEN DEATH|          8|       Elderly|
|       160146198|    01/23/2016|  3|1.0|   0|      NUL

In [ ]:
diagnoses_dict = {41: 'Ingestion',
 42: 'Aspiration',
 46: 'Burns, Electrical',
 47: 'Burns, Not Specified',
 48: 'Burns, Scald',
 49: 'Burns, Chemical',
 50: 'Amputaion',
 51: 'Burns, Thermal',
 52: 'Concussions',
 53: 'Contusions, Abrasions',
 54: 'Crushing',
 55: 'Dislocation',
 56: 'Foreign Body',
 57: 'Fracture',
 58: 'Hematoma',
 59: 'Laceration',
 60: 'Dental Injury',
 61: 'Nerve Damage',
 62: 'Internal Organ Injury',
 63: 'Puncture',
 64: 'Strain, Sprain',
 65: 'Anoxia',
 66: 'Hemorrhage',
 67: 'Electric Shock',
 68: 'Poisoning',
 69: 'Submersion',
 71: 'Other/Not Stated',
 72: 'Avulsion',
 73: 'Burns, Radiation',
 74: 'Dermatitis, Conjunctivitis'}

time: 860 µs (started: 2025-03-23 14:51:13 +00:00)


Get counts of distinct diagnosis values using mapreduce

In [ ]:
df2.groupby('Diagnosis').count().sort('count', ascending=False).show()

+---------+-----+
|Diagnosis|count|
+---------+-----+
|       71|  328|
|       62|  104|
|       69|   52|
|       65|   37|
|       57|   23|
|       59|   14|
|       51|   14|
|       53|    8|
|       68|    6|
|       58|    2|
|       42|    2|
|       66|    2|
|       67|    2|
|       52|    1|
|       54|    1|
|       48|    1|
|       72|    1|
|       63|    1|
+---------+-----+

time: 4.59 s (started: 2025-03-23 14:51:15 +00:00)


Age Analysis

In [ ]:
df2.groupby('Age_Categories').count().sort('count', ascending=False).show()

+--------------+-----+
|Age_Categories|count|
+--------------+-----+
|       Elderly|  255|
|         Adult|   99|
|      Very old|   91|
|        Infant|   66|
|         Child|   54|
|    Adolescent|   27|
|  Not Recorded|    7|
+--------------+-----+

time: 3.29 s (started: 2025-03-23 14:51:25 +00:00)


In [ ]:
age_dfs = [df2[df2['Age_Categories'] == x] for x in age_dict.values()]

time: 56.8 ms (started: 2025-03-23 14:51:29 +00:00)


In [49]:
%%memit
rdds = []
for i in range(len(age_dfs)):
  rdd = age_dfs[i].rdd
  rdds.append(rdd.map(lambda x : (x['Diagnosis'], 1)).reduceByKey(lambda a,b: a + b).map(lambda x:(diagnoses_dict.get(x[0]), x[1])).sortBy(lambda x: x[1], ascending=False))

peak memory: 423.98 MiB, increment: 0.19 MiB
time: 23.8 s (started: 2025-03-23 15:07:21 +00:00)


In [50]:
%%memit
for i in range(len(rdds)):
  output = rdds[i].collect()
  print("Age group: %s" %list(age_dict.values())[i])
  for diagnosis, count in output:
    print("{}: {}".format(diagnosis, count))
  print("\n")

Age group: Not Recorded
Other/Not Stated: 4
Anoxia: 2
Avulsion: 1


Age group: Child
Submersion: 22
Other/Not Stated: 11
Anoxia: 10
Internal Organ Injury: 5
Hemorrhage: 2
Burns, Thermal: 2
Fracture: 1
Electric Shock: 1


Age group: Adolescent
Other/Not Stated: 9
Internal Organ Injury: 8
Submersion: 7
Anoxia: 2
Fracture: 1


Age group: Adult
Other/Not Stated: 49
Internal Organ Injury: 13
Submersion: 8
Anoxia: 8
Fracture: 5
Laceration: 5
Poisoning: 4
Burns, Thermal: 3
Concussions: 1
Crushing: 1
Electric Shock: 1
Puncture: 1


Age group: Elderly
Other/Not Stated: 163
Internal Organ Injury: 47
Burns, Thermal: 8
Anoxia: 7
Fracture: 7
Submersion: 7
Laceration: 6
Contusions, Abrasions: 6
Hematoma: 2
Poisoning: 2


Age group: Very old
Other/Not Stated: 42
Internal Organ Injury: 30
Fracture: 9
Laceration: 3
Contusions, Abrasions: 2
Submersion: 2
Aspiration: 1
Burns, Thermal: 1
Anoxia: 1


Age group: Infant
Other/Not Stated: 50
Anoxia: 7
Submersion: 6
Internal Organ Injury: 1
Aspiration: 1
Burns

Race output

In [ ]:
passed_away = df2.filter(df['Disposition'] == 8)
passed_away.head()

Row(CPSC_Case_Number='160122220', Treatment_Date='01/08/2016', Age=84, Sex=1.0, Race=0, Other_Race=None, Hispanic=None, Body_Part=85, Diagnosis=71, Other_Diagnosis='CARDIAC ARREST', Disposition=8, Age_Categories='Very old')

time: 324 ms (started: 2025-03-23 14:54:38 +00:00)


In [ ]:
passed_away.groupBy('Diagnosis').count().orderBy('count', ascending=False).show()

+---------+-----+
|Diagnosis|count|
+---------+-----+
|       71|  328|
|       62|  104|
|       69|   52|
|       65|   37|
|       57|   23|
|       59|   14|
|       51|   14|
|       53|    8|
|       68|    6|
|       58|    2|
|       42|    2|
|       66|    2|
|       67|    2|
|       52|    1|
|       54|    1|
|       48|    1|
|       72|    1|
|       63|    1|
+---------+-----+

time: 2.75 s (started: 2025-03-23 14:54:39 +00:00)


In [ ]:
# white
white_df = passed_away.filter(passed_away["Race"] == 1.0)

# black
black_df = passed_away.filter(passed_away["Race"] == 2.0)

# asian
asian_df = passed_away.filter(passed_away["Race"] == 4.0)

# native
native_df = passed_away.filter(passed_away["Race"] == 5.0)

# islander
islander_df = passed_away.filter(passed_away["Race"] == 6.0)


time: 50.7 ms (started: 2025-03-23 14:54:43 +00:00)


In [51]:
%%memit
rdds = []
rdd = white_df.rdd
rdds.append(rdd.map(lambda x : (x['Diagnosis'], 1)).reduceByKey(lambda a,b: a + b).map(lambda x:(diagnoses_dict.get(x[0]), x[1])).sortBy(lambda x: x[1], ascending=False))
rdd = black_df.rdd
rdds.append(rdd.map(lambda x : (x['Diagnosis'], 1)).reduceByKey(lambda a,b: a + b).map(lambda x:(diagnoses_dict.get(x[0]), x[1])).sortBy(lambda x: x[1], ascending=False))
rdd = asian_df.rdd
rdds.append(rdd.map(lambda x : (x['Diagnosis'], 1)).reduceByKey(lambda a,b: a + b).map(lambda x:(diagnoses_dict.get(x[0]), x[1])).sortBy(lambda x: x[1], ascending=False))
rdd = native_df.rdd
rdds.append(rdd.map(lambda x : (x['Diagnosis'], 1)).reduceByKey(lambda a,b: a + b).map(lambda x:(diagnoses_dict.get(x[0]), x[1])).sortBy(lambda x: x[1], ascending=False))
rdd = islander_df.rdd
rdds.append(rdd.map(lambda x : (x['Diagnosis'], 1)).reduceByKey(lambda a,b: a + b).map(lambda x:(diagnoses_dict.get(x[0]), x[1])).sortBy(lambda x: x[1], ascending=False))

peak memory: 424.22 MiB, increment: 0.00 MiB
time: 17.8 s (started: 2025-03-23 15:10:33 +00:00)


In [52]:
%%memit
race_dict = {1.0 : 'White', 2.0 : 'Black', 3.0:'Asian', 4.0: 'Native', 5.0: 'Islander'}
for i in range(len(rdds)):
  output = rdds[i].collect()
  print("Race: %s" %list(race_dict.values())[i])
  for diagnosis, count in output:
    print("{}: {}".format(diagnosis, count))
  print("\n")

Race: White
Other/Not Stated: 91
Internal Organ Injury: 30
Submersion: 16
Anoxia: 11
Laceration: 9
Fracture: 7
Burns, Thermal: 6
Contusions, Abrasions: 4
Poisoning: 2
Hemorrhage: 1
Crushing: 1
Aspiration: 1
Electric Shock: 1
Puncture: 1


Race: Black
Other/Not Stated: 40
Anoxia: 13
Internal Organ Injury: 9
Submersion: 7
Burns, Thermal: 4
Laceration: 2
Avulsion: 1
Contusions, Abrasions: 1


Race: Asian
Other/Not Stated: 5
Internal Organ Injury: 3
Submersion: 2


Race: Native
Fracture: 1


Race: Islander


peak memory: 424.24 MiB, increment: 0.02 MiB
time: 2.74 s (started: 2025-03-23 15:11:22 +00:00)


Gender Analysis


In [38]:
male_df = passed_away.filter(passed_away["Sex"] == 1.0)
female_df = passed_away.filter(passed_away["Sex"] == 2.0)

time: 19.3 ms (started: 2025-03-23 15:03:14 +00:00)


In [53]:
%%memit
rdds = []
rdd = male_df.rdd
rdds.append(rdd.map(lambda x : (x['Diagnosis'], 1)).reduceByKey(lambda a,b: a + b).map(lambda x:(diagnoses_dict.get(x[0]), x[1])).sortBy(lambda x: x[1], ascending=False))
rdd = female_df.rdd
rdds.append(rdd.map(lambda x : (x['Diagnosis'], 1)).reduceByKey(lambda a,b: a + b).map(lambda x:(diagnoses_dict.get(x[0]), x[1])).sortBy(lambda x: x[1], ascending=False))

peak memory: 424.27 MiB, increment: 0.01 MiB
time: 6.25 s (started: 2025-03-23 15:11:56 +00:00)


In [55]:
%%memit
sex_dict = {1.0 : 'Male', 2.0 : "Female"}
for i in range(len(rdds)):
  output = rdds[i].collect()
  print("Gender: %s" %list(sex_dict.values())[i])
  for diagnosis, count in output:
    print("{}: {}".format(diagnosis, count))
  print("\n")

Gender: Male
Other/Not Stated: 233
Internal Organ Injury: 70
Submersion: 40
Anoxia: 20
Fracture: 14
Laceration: 13
Burns, Thermal: 12
Poisoning: 5
Contusions, Abrasions: 4
Aspiration: 2
Electric Shock: 2
Hemorrhage: 1
Crushing: 1
Burns, Scald: 1
Avulsion: 1
Puncture: 1


Gender: Female
Other/Not Stated: 95
Internal Organ Injury: 34
Anoxia: 17
Submersion: 12
Fracture: 9
Contusions, Abrasions: 4
Hematoma: 2
Burns, Thermal: 2
Concussions: 1
Hemorrhage: 1
Poisoning: 1
Laceration: 1


peak memory: 424.30 MiB, increment: 0.00 MiB
time: 917 ms (started: 2025-03-23 15:12:55 +00:00)
